In [8]:
# copied from: https://www.geeksforgeeks.org/get-post-requests-using-python/
# importing the requests library 
import requests 

# defining the api-endpoint 
API_ENDPOINT = 'http://nlu:8080/factextraction/analyze'
headers = {"accept": "application/json", "content-type": "application/json"}


# data to be sent to api 
data = r'{"docId": "doc2", "text":"Plastic, Copper and metal are materials. Some viruses may stick on it.", "extractConcepts": "true", "language": "en" }'

# sending post request and saving response as response object 
r = requests.post(url = API_ENDPOINT, data = data, headers = headers) 



In [9]:
# extracting response text 
response_text = r.text 
response_text

'{"docId":"doc2","language":"en","matches":[{"charLength":7,"charOffset":0,"text":"Plastic","entity":{"id":"http://www.wikidata.org/entity/Q11474","confidence":0.775866761312668}},{"charLength":6,"charOffset":9,"text":"Copper","entity":{"id":"http://www.wikidata.org/entity/Q753","confidence":0.8870102642876663}},{"charLength":5,"charOffset":20,"text":"metal","entity":{"id":"http://www.wikidata.org/entity/Q11426","confidence":0.5593303515943758}},{"charLength":9,"charOffset":30,"text":"materials","entity":{"id":"http://www.wikidata.org/entity/Q16829513","confidence":0.212667514858712}},{"charLength":7,"charOffset":46,"text":"viruses","entity":{"id":"http://www.wikidata.org/entity/Q808","confidence":0.5377329609395741}}],"facts":[{"subject":{"text":"Plastic Copper and metal","charOffset":0,"charLength":25},"relation":{"text":"are","charOffset":26,"charLength":3},"object":{"text":"materials","charOffset":30,"charLength":9}},{"subject":{"text":"Some viruses","charOffset":41,"charLength":12

In [19]:
# the response will be something like this:
response = json.loads('{"docId":"doc2","language":"en","matches":[{"charLength":7,"charOffset":0,"text":"Plastic","entity":{"id":"http://www.wikidata.org/entity/Q11474","confidence":0.775866761312668}},{"charLength":6,"charOffset":9,"text":"Copper","entity":{"id":"http://www.wikidata.org/entity/Q753","confidence":0.8870102642876663}},{"charLength":5,"charOffset":20,"text":"metal","entity":{"id":"http://www.wikidata.org/entity/Q11426","confidence":0.5593303515943758}},{"charLength":9,"charOffset":30,"text":"materials","entity":{"id":"http://www.wikidata.org/entity/Q16829513","confidence":0.212667514858712}},{"charLength":7,"charOffset":46,"text":"viruses","entity":{"id":"http://www.wikidata.org/entity/Q808","confidence":0.5377329609395741}}],"facts":[{"subject":{"text":"Plastic Copper and metal","charOffset":0,"charLength":25},"relation":{"text":"are","charOffset":26,"charLength":3},"object":{"text":"materials","charOffset":30,"charLength":9}},{"subject":{"text":"Some viruses","charOffset":41,"charLength":12},"relation":{"text":"may stick","charOffset":54,"charLength":9},"object":{"text":"on it","charOffset":64,"charLength":5}}],"entities":[{"id":"http://www.wikidata.org/entity/Q753","name":"Copper","url":"http://en.wikipedia.org/wiki/Copper","type":"CONCEPT","salience":0.0},{"id":"http://www.wikidata.org/entity/Q11474","name":"Plastic","url":"http://en.wikipedia.org/wiki/Plastic","type":"CONCEPT","salience":0.0},{"id":"http://www.wikidata.org/entity/Q16829513","name":"Werkstoff","url":"http://de.wikipedia.org/wiki/Werkstoff","type":"CONCEPT","salience":0.0},{"id":"http://www.wikidata.org/entity/Q11426","name":"Metal","url":"http://en.wikipedia.org/wiki/Metal","type":"CONCEPT","salience":0.0},{"id":"http://www.wikidata.org/entity/Q808","name":"Virus","url":"http://en.wikipedia.org/wiki/Virus","type":"CONCEPT","salience":0.0}]}')
import json


# all requests will be json docs we sum them up
responses = [response]

response['docId']


'doc2'

In [ ]:
# we can ingest all data into our simple knowlegegraph, but also do some lookups to wikidata before.

In [4]:
!pip install rdflib SPARQLWrapper

In [2]:
# we also want to get more information attached to the data:
# E.g. Copper is this one: http://www.wikidata.org/entity/Q11474
# we ask wikidata to provide also more information to this entity as a knowledgegraph

import rdflib

g = rdflib.Graph()
result = g.parse("http://www.wikidata.org/entity/Q11474")

print("graph has %s statements." % len(g))

graph has 15774 statements.


In [ ]:
# we should shift the following against a local graph. Maybe we want to just download all P31, P275 and labels to a local graph and then throw the data toegther?

In [33]:
from rdflib import Graph, URIRef, Namespace
wdp = Namespace('http://www.wikidata.org/prop/direct/')
rdfs = Namespace('http://www.w3.org/2000/01/rdf-schema#')
graph = Graph('SPARQLStore', identifier="https://query.wikidata.org/")


def initialize_wikidata_graph():
    graph.open("https://query.wikidata.org/sparql")


def get_instance_of_from_wikidata(IRI):
    instance_of = graph.value(
        URIRef(IRI),
        wdp.P31)
    return instance_of


def get_subclass_of_from_wikidata_as_list(IRI):
    subclass_of_list = graph.objects(
        URIRef(IRI),
        wdp.P279)
    return subclass_of_list


def get_label_to_thing(IRI):
    label_to = graph.preferredLabel(
        URIRef(IRI),
         lang='en')
    return label_to[0][1]


initialize_wikidata_graph()

instance = get_instance_of_from_wikidata("http://www.wikidata.org/entity/Q11474")

print("instance of " , instance, " and label ", get_label_to_thing(instance))


for subclass_of in get_subclass_of_from_wikidata_as_list("http://www.wikidata.org/entity/Q11474"):
    print("subclass of ", subclass_of, " and label: ", get_label_to_thing(subclass_of))
    

instance of  http://www.wikidata.org/entity/Q1783121  and label  organic matter


HTTPError: HTTP Error 429: Too Many Requests